# 18.7 Takeaway - Transform one String to another

Given a Set `D` filled with strings of the same length, and two strings `s` and `t` that also have the same length, compute the length of the shortest sequence to turn `s` into `t` using strings in `D`. Each transformation involves changing 1 letter at a time.

## Example

```
D = {bat, cot, dog, dag, dot, cat}
s = cat
t = dog
```

Output: 3 because this would be the sequence {cat, cot, dot, dog}

## Brainstorming - Session 1 Naiive

- Thinking about this problem, it's evident that it would be way too expensive to try every combination and go down every path.
- Considering the word path, this can be similar to a graph problem if we consider the vertices as the words in the dictionary, and the edges as words that are a difference of 1 to those vertices. It's a good idea to take a graph approach to this problem.
- Since the problem asks for a shortest path aka an optimization, we should likely use BFS to get there.
- Let's start by building the graph

![](../%20images/string_transform_example_graph.png)

  - To do this, we need to create a vertex out of every item in D. 
  - Then we need to compare against every other word to see if the edit distance = 1. likely with a double for loop (O(D^2) runtime)
- After the graph is built, we can do a simple BFS shortest path algorithm to get there
  - Have a queue starting the (s, 0), 0 representing how many transformations we've had to make
  - Every time we pop a node and go through it's neighbor strings, we need to append (neighbor of s, +1) to the queue
  - When the node popped = string `t`, return the count
- If we reach the end of the queue, return -1 because it wasn't found to be possible to reach `t`

## Implementation - Naiive

In [37]:
from collections import deque

def transform_string_naiive(D, s, t):
    def build_graph(D):
        graph = {vertex: [] for vertex in D}
        for word1 in D:
            for word2 in D:
                if word1 == word2:
                    continue
                diff, i = 0, 0
                while i < len(word1) and diff < 2:
                    if word1[i] != word2[i]:
                        diff += 1
                    i += 1
                
                if diff == 1:
                    graph[word1].append(word2)
        return graph

    graph = build_graph(D)
    queue = deque([(s, 0)])
    visited = set()
    while queue:
        vertex, count = queue.popleft()
        if vertex in visited:
            continue   
        visited.add(vertex)

        if vertex == t:
            return count

        for neighbor in graph[vertex]:
            new_vertex = (neighbor, count + 1)
            queue.append(new_vertex)

    return -1

In [38]:
D = {'bat', 'cot', 'dog', 'dag', 'dot', 'cat'}
s = 'cat'
t = 'dog'
result = transform_string_naiive(D, s, t)
print(result)

3


## Brainstorming - Session 2 Optimized

- Realizing that this was too slow because of the massive amount of checks, and the fact that we aren't really taking advantage of the Set we're given
- It turns out building our path as we go is a much more efficient way to go. 
- Starting from our first word s popped out of the queue
  - We need to explore all other 1 diff words that exist in the set. This takes advantage of the set's properties with its O(1) lookup time, and there's a finite amount of 1 letter diff words we can make. Roughly (O(len of word*number of ascii letters)).

In [39]:
from collections import deque
import string 
def transform_string(D, s, t):
    queue = deque([(s, 0)])
    visited = set()

    while queue:
        word, count = queue.popleft()
        if word == t:
            return count
        
        for i in range(len(word)):
            # Grabs all ascii lowercase letters
            for letter in string.ascii_lowercase:
                new_word = word[:i] + letter + word[i+1:]
                if new_word not in visited and new_word in D:
                    visited.add(new_word)
                    queue.append((new_word, count + 1))

    return -1

In [40]:
D = {'bat', 'cot', 'dog', 'dag', 'dot', 'cat'}
s = 'cat'
t = 'dog'
result = transform_string(D, s, t)
print(result)

3


# Analysis

Runtime: Still O(d^2) at worst case to process all different possible edges 
Space: O(d) considering each word ends up in the queue one time max. Much better than d^2 for creating a whole initial graph in the naiive approach